In [147]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [148]:
folder = '../data/raw_data/'

df_categories = pd.read_csv(f'{folder}df_categories.csv')
df_comments = pd.read_csv(f'{folder}df_comments_video.csv')
df_transcript = pd.read_csv(f'{folder}df_transcript_original.csv')
df_videos = pd.read_csv(f'{folder}df_video_data.csv')
df_search = pd.read_csv(f'{folder}df_search_info.csv')
df_channel = pd.read_csv(f'{folder}df_channel_data.csv')

/tmp/ipykernel_8100/1966000850.py:4: DtypeWarning: Columns (5,8,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_comments = pd.read_csv(f'{folder}df_comments_video.csv')


# Clean df_channel_data

In [149]:
df_channel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639 entries, 0 to 638
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   channel_id        639 non-null    object 
 1   title             639 non-null    object 
 2   description       578 non-null    object 
 3   published_at      638 non-null    object 
 4   subscriber_count  638 non-null    float64
 5   video_count       637 non-null    float64
 6   view_count        637 non-null    float64
 7   region            537 non-null    object 
dtypes: float64(3), object(5)
memory usage: 40.1+ KB


In [150]:
# Drop wrong fetch data 
df_channel.dropna(subset=['view_count'], inplace=True)

# Change to datetime
df_channel['published_at'] = pd.to_datetime(df_channel['published_at'], format='mixed')

# Export cleaned data
df_channel.to_csv('../data/df_channel_data.csv', index=False)

# Show info cleaned
df_channel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 637 entries, 0 to 638
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   channel_id        637 non-null    object             
 1   title             637 non-null    object             
 2   description       576 non-null    object             
 3   published_at      637 non-null    datetime64[ns, UTC]
 4   subscriber_count  637 non-null    float64            
 5   video_count       637 non-null    float64            
 6   view_count        637 non-null    float64            
 7   region            537 non-null    object             
dtypes: datetime64[ns, UTC](1), float64(3), object(4)
memory usage: 44.8+ KB


# Clean df_video_data

In [151]:
df_videos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1791 entries, 0 to 1790
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      1791 non-null   int64 
 1   videoId         1791 non-null   object
 2   title           1791 non-null   object
 3   channelId       1791 non-null   object
 4   description     1553 non-null   object
 5   publishedAt     1791 non-null   object
 6   thumbnail_url   1791 non-null   object
 7   tags            1791 non-null   object
 8   live_broadcast  1791 non-null   object
 9   categoryId      1791 non-null   int64 
 10  viewCount       1791 non-null   int64 
 11  likeCount       1791 non-null   int64 
 12  commentCount    1791 non-null   int64 
 13  licensed        1791 non-null   bool  
 14  duration        1791 non-null   object
 15  caption         1791 non-null   bool  
dtypes: bool(2), int64(5), object(9)
memory usage: 199.5+ KB


In [152]:
import isodate

# Remove duplicated video 
df_videos.drop_duplicates(subset=['videoId'], keep='first', inplace=True)

# Convert ISO 8601 duration to a readable format (timedelta)
def convert_duration(duration_str):
    duration = isodate.parse_duration(duration_str)
    return duration.seconds

# Change duration to total seconds
df_videos['duration'] = df_videos['duration'].apply(convert_duration)

# Drop live broadcast column and Unnamed 0
df_videos.drop(columns=['Unnamed: 0','live_broadcast'], inplace=True)

# Export the cleaned data
df_videos.to_csv('../data/df_video_data.csv', index=False)

df_videos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1431 entries, 0 to 1790
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   videoId        1431 non-null   object
 1   title          1431 non-null   object
 2   channelId      1431 non-null   object
 3   description    1259 non-null   object
 4   publishedAt    1431 non-null   object
 5   thumbnail_url  1431 non-null   object
 6   tags           1431 non-null   object
 7   categoryId     1431 non-null   int64 
 8   viewCount      1431 non-null   int64 
 9   likeCount      1431 non-null   int64 
 10  commentCount   1431 non-null   int64 
 11  licensed       1431 non-null   bool  
 12  duration       1431 non-null   int64 
 13  caption        1431 non-null   bool  
dtypes: bool(2), int64(5), object(7)
memory usage: 148.1+ KB


# Clean df_comments_video

In [153]:
df_comments.info()
pd.isnull(df_comments).sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229246 entries, 0 to 229245
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   comment_id         229246 non-null  object 
 1   kind               229245 non-null  object 
 2   author             229035 non-null  object 
 3   author_channel_id  229245 non-null  object 
 4   text               229203 non-null  object 
 5   like_count         229244 non-null  object 
 6   published_at       229244 non-null  object 
 7   updated_at         229244 non-null  object 
 8   can_rate           229244 non-null  object 
 9   viewer_rating      229244 non-null  object 
 10  can_reply          229243 non-null  object 
 11  total_reply_count  229243 non-null  float64
 12  is_public          229243 non-null  object 
 13  video_id           229243 non-null  object 
dtypes: float64(1), object(13)
memory usage: 24.5+ MB


comment_id             0
kind                   1
author               211
author_channel_id      1
text                  43
like_count             2
published_at           2
updated_at             2
can_rate               2
viewer_rating          2
can_reply              3
total_reply_count      3
is_public              3
video_id               3
dtype: int64

In [154]:
# Drop unnecessary data
df_comments.drop(columns=['viewer_rating', 'can_rate', 'can_reply', 'is_public', 'kind'], inplace=True)

# Drop na values
df_comments.dropna(inplace=True)

# Transform data to correct type
df_comments['like_count'] = df_comments['like_count'].astype('float64')
df_comments['published_at'] = pd.to_datetime(df_comments['published_at'])
df_comments['updated_at'] = pd.to_datetime(df_comments['updated_at'])

# Export cleaned data
df_comments.to_csv('../data/df_comments_video.csv', index=False)

df_comments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 228991 entries, 0 to 229245
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype              
---  ------             --------------   -----              
 0   comment_id         228991 non-null  object             
 1   author             228991 non-null  object             
 2   author_channel_id  228991 non-null  object             
 3   text               228991 non-null  object             
 4   like_count         228991 non-null  float64            
 5   published_at       228991 non-null  datetime64[ns, UTC]
 6   updated_at         228991 non-null  datetime64[ns, UTC]
 7   total_reply_count  228991 non-null  float64            
 8   video_id           228991 non-null  object             
dtypes: datetime64[ns, UTC](2), float64(2), object(5)
memory usage: 17.5+ MB


1202

# Clean df_transcript

In [155]:
df_transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670 entries, 0 to 1669
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           1670 non-null   object
 1   original_language  1670 non-null   object
 2   transcription      1670 non-null   object
dtypes: object(3)
memory usage: 39.3+ KB


In [156]:
# Drop duplicated values
df_transcript.drop_duplicates(subset=['video_id'], inplace=True)

# Cleaned language values
def get_cleaned_name(str):
    return str.split(' ')[0].strip().lower()

df_transcript['language'] = df_transcript['original_language'].apply(get_cleaned_name)

# Export dataframe
df_transcript.to_csv('../data/df_transcript_original.csv')

df_transcript.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1343 entries, 0 to 1669
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   video_id           1343 non-null   object
 1   original_language  1343 non-null   object
 2   transcription      1343 non-null   object
 3   language           1343 non-null   object
dtypes: object(4)
memory usage: 52.5+ KB


# Normalize text by language

In [157]:
from langdetect import detect
from nltk.corpus import stopwords

# Create a column with language detected
df_transcript['language_detect'] = df_transcript['transcription'].apply(detect)

In [ ]:
df_transcript['language']

0         hindi
1       english
2       english
3       english
4       english
         ...   
1665    english
1666    english
1667    english
1668      hindi
1669    english
Name: language, Length: 1343, dtype: object

In [ ]:
# Remove specific stopwords from each langua
def remove_stopwords(text, language):
    stop_words = set(stopwords.words(language))
    return ' '.join([word for word in text.split() if word not in stop_words])

df_transcript['transcription_normalized'] = df_transcript[df_transcript['language'] == 'english'].apply(lambda x: remove_stopwords(x['transcription'], x['language']), axis=1)

In [ ]:
df_transcript['transcription_normalized']

0                                                     NaN
1       ten years ago samsung released trailer concept...
2       smartphone market 2023 definitely lows highs t...
3       [Music] right welcome smartphone Awards 2023 l...
4       year finally shaken smartphone markets we've n...
                              ...                        
1665    okay i've spent last two weeks announced flags...
1666    [Music] hey set see best gaming smartphones gr...
1667    what's going everybody kalupustek coming back ...
1668                                                  NaN
1669    Engish cont mobile Poco X6 Pro Mobile 6.67 1.5...
Name: transcription_normalized, Length: 1343, dtype: object